# Новосибирск

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
%matplotlib inline

In [2]:
DF = pd.read_excel('prodoctor3quart.xlsx', dtype={'Номера':str})


In [6]:
def join_list(lst):
    return ', '.join([str(i) for i in lst])



au_telefons = pd.read_excel(r'C:\Users\AmanovRA\Текущее2\РК_3_квартал\НСК\НСК_оборот\НСК_оборот.xlsx')

dom, mobil, work = au_telefons['Телефон домашний'].tolist(), au_telefons['Телефон мобильный'].tolist(), au_telefons['Телефон рабочий'].tolist()

pattern = r'\(\d{2}\)'
pattern2 = r'[а-яА-Яa-zA-Z]{1}'

for key in (dom, mobil, work):
    for i, j in enumerate(key):
        if type(j) == str:
            key[i] = ''.join([x if x not in ',;!:.\\/+' else ' ' for x in j])        
            key[i] = key[i].lstrip(' ')
            key[i] = key[i].rstrip(' ')
            if re.search(pattern2, key[i]):
                key[i] = re.sub(pattern2, '', key[i])
            key[i] = ''.join([x for x in key[i] if x not in '()-'])  
            
for key in (dom, mobil, work):
    for i, j in enumerate(key):
        if type(j) == str:
            if len(j) == 7:
                key[i] = '7812' + key[i]
            elif len(j) == 11 and j[0] == '8':
                key[i] = '7' + j[1:]
            elif len(j) == 10:
                key[i] = '7' + j
                
def concate_lists(*args) -> list:   # соединим все номера в одну строку чтобы проще было искать
    temp_lst = []
    for i in range(len(args[0])):
        temp_lst.append(' '.join([str(x[i]) for x in args]))
    return temp_lst
            
lst_search = concate_lists(dom, mobil, work)                 

lst_with_coincidenc_telefon  = []
for index, row in enumerate(lst_search):
    lst_with_coincidenc_telefon.append('')
    for i in DF['Номера']:
        if str(i) in row:
            lst_with_coincidenc_telefon[index] = str(i)
            break
            
au_telefons['Совпадение'] = lst_with_coincidenc_telefon            
            
au_telefons_final = au_telefons[au_telefons['Совпадение'] != '']
au_telefons_final = au_telefons_final.merge(DF, left_on='Совпадение', right_on='Номера', how='left')
au_telefons_final = au_telefons_final.fillna('')


# не забыть проверить чтобы даты приемов были больше или равны дате звонка          



In [7]:
au_telefons_final['Совпадение дат'] = au_telefons_final['Дата приема'] >= au_telefons_final['Дата']

au_telefons_final = au_telefons_final[au_telefons_final['Совпадение дат'] == True]

au_telefons_final.to_excel('НСК_обработано.xlsx')